## Workflow for matching and importing location data from iDAI.gazetteer to Wikidata

This notebook describes the workflow used in FAIR.rdm for matching location data in iDAI.gazetteer with Wikidata.

The Python code supports the creation of new locations using quick statements.

---

### Workflow zum Abgleich und Import von Ortsdaten aus iDAI.gazetteer zu Wikidata

Das Notebook beschreibt den in FAIR.rdm praktizierten Workflow zum Abgleich von Ortsdaten im iDAI.gazetteer mit Wikidata.

Der Python-Code unterstützt beim Anlegen neuer Orte mittels Quick Statements.

In [ ]:
#import necessary libraries

import pandas as pd
import requests
import time

First, [OpenRefine](https://openrefine.org/) must be used to compare which data is already in Wikidata. The Wikidata URIs must then be specified in a separate column (Compare/Add columns with URLs of matched entities).

This table can then be exported for the following code.

Further information: https://www.wikidata.org/wiki/Wikidata:Tools/OpenRefine

---

Als erstes muss mit [OpenRefine](https://openrefine.org/) ein Agleich durchgeführt werden, welche Daten sich bereits in Wikidata befinden. Die Wikidata URIs sind dann in einer separaten Spalten anzugeben (Abgleichen/Add columns with URLs of matched entities).

Diese Tablle kann dann für den folgenden Code exportiert werden.

Weitere Informationen: https://www.wikidata.org/wiki/Wikidata:Tools/OpenRefine/de

In [ ]:
df_toimport = pd.read_excel("Gazetteer_To_Wikidata_Mapping_2025-07-11.xlsx", sheet_name="wikidata")
df_toimport = df_toimport.fillna("absent")

df_toimport = df_toimport[df_toimport['Wikidata_URL'] == "absent"]
df_toimport = df_toimport.reset_index(drop=True)
df_toimport.drop('Wikidata_URL', axis=1, inplace=True)

#Test with a subset of the data:
#df_test = df_toimport[14:16]

In [ ]:
#If the first column contains Gazetteer URIs, the Gazetteer ID is extracted here.
def get_gaz_id(uri):
    parts = uri.split("/")
    gazid = parts[-1]
    return gazid

In [ ]:
#This function retrieves the name of a place from the Gazetteer.
def get_gaz_title(gaz_id):
    gaz_id = gaz_id.split("/")[-1]
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'prefName' in data:
            title = data['prefName']['title']
            return title
        else:
            title = "ERROR_no_title!!!"
            return title

In [ ]:
#This function takes the name of a place and sets it as the Wikidata label.
def get_label(row):
    label = row['PlaceName']
    return label

In [ ]:
# This function retrieves the type of a place from the Gazetteer and sets it as the Wikidata description. It may needs to be extended.

def get_gaz_type(gaz_id):
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        types = data['types']
        title = data['prefName']['title']
        if "Keramik" in title:
            type = "archäologischer Keramikstil"
        elif 'archaeological-area' in types:
            type = "Archäologischer Bereich"
        elif 'archaeological-site' in types:
            type = "Archäologischer Ort"
        else:
            type = types[0]
        return type
    else:
        type = "ERROR_no_type!!!"
        return type

In [ ]:
#This function determines the statement "is a ..." (P31) for Wikidata.

def get_p31(description):
    if description == "Archäologischer Bereich":
        p31 = "Q839954 (Archäologische Stätte)"
        return p31
    elif description == "archäologischer Keramikstil":
        p31 = "Q24017852 (Keramikstil)"
        return p31
    elif description == "Archäologischer Ort":
        p31 = "Q839954 (Archäologische Stätte)"
        return p31
    else:
        p31 = "set manually"
        return p31

In [ ]:
#If the coordinates are polygons, this function calculates the centroid of the polygon and imports it into Wikidata.

def polygon_centroid(coords):
    # Unwrap the nested list structure
    points = coords[0][0]
    x_list = [p[0] for p in points]
    y_list = [p[1] for p in points]
    n = len(points) - 1  # last point is same as first

    area = 0.0
    cx = 0.0
    cy = 0.0

    for i in range(n):
        factor = (x_list[i] * y_list[i+1] - x_list[i+1] * y_list[i])
        area += factor
        cx += (x_list[i] + x_list[i+1]) * factor
        cy += (y_list[i] + y_list[i+1]) * factor

    area *= 0.5
    if area == 0:
        # fallback: average of points
        return (sum(x_list)/len(x_list), sum(y_list)/len(y_list))
    cx /= (6.0 * area)
    cy /= (6.0 * area)
    return (cx, cy)

In [ ]:
#This function retrieves the latitude of a place from the Gazetteer.

def get_latitude(gaz_id):
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print(data)
        location = data['prefLocation']
        if 'coordinates' in location:
            lat = data['prefLocation']["coordinates"][0]
        elif 'shape' in location:
            coords = location['shape']
            center = polygon_centroid(coords)
            lat = center[0]
        else:
            lat = "ERROR_no_coordinates!!!"
        return lat

In [ ]:
#This function retrieves the longitude of a place from the Gazetteer.

def get_longitude(gaz_id):
    url = f"https://gazetteer.dainst.org/doc/{gaz_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'prefLocation' in data and 'coordinates' in data['prefLocation']:
            long = data['prefLocation']["coordinates"][-1]
        elif 'shape' in data['prefLocation']:
            coords = data['prefLocation']['shape']
            center = polygon_centroid(coords)
            long = center[1]
        else:
            long = "ERROR_no_coordinates!!!"
        return long

In [ ]:
#This function determines the country of a place based on its latitude and longitude using the Nominatim service from OpenStreetMap.

def get_country_from_latlon(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lon,
        "lon": lat,
        "format": "json",
        "zoom": 3,  # 3 = country level
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; CopilotBot/1.0)"
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        address = data.get("address", {})
        country = address.get("country")
        return country
    else:
        return None

In [ ]:
#Main Loop

labels = []
descriptions = []
type = []
lat = []
long = []
states = []

for index, row in df_toimport.iterrows():
    label = get_label(row)
    labels.append(label)
    gaz_id = get_gaz_id(row['GazetteerID'])
    description = get_gaz_type(gaz_id)
    descriptions.append(description)
    p31_value = get_p31(description)
    type.append(p31_value)
    latitude = get_latitude(gaz_id)
    longitude = get_longitude(gaz_id)
    #latitude and longitude should only have five decimal places
    latitude = round(latitude, 5)
    longitude = round(longitude, 5)
    lat.append(latitude)
    long.append(longitude)
    country = get_country_from_latlon(latitude, longitude)
    states.append(country)
    #Wait for three seconds to avoid rate limiting
    time.sleep(3)

print(labels)
print(descriptions)
print(type)
print(lat)
print(long)
print(states)

In [ ]:
#export to csv
df_toexport = pd.DataFrame({
    'GazetteerID': df_toimport['GazetteerID'],
    'Label': labels,
    'Description': descriptions,
    'Type': type,
    'Latitude': lat,
    'Longitude': long,
    'Country': states
})

df_toexport.to_csv("Wikidata_information.csv", index=False, encoding='utf-8', sep=';')

Now you can make the necessary changes to the CSV file and add entries manually.

Then you can generate the QuickStatements using the following code:

---

Jetzt können an der CSV nötige Änderungen vorgenommen werden und Einträge manuell ergänzt werden.

Dananch können mit folgendem Code die QuickStatements erzeugt werden:

In [ ]:
#The mapping of countries to Wikidata Q-IDs may need to be extended.

country_mapping = {
    "ليبيا": "Q1016",   # Libyen
    "Sudan": "Q1049",
    "مصر": "Q79",       # Ägypten
    "Niger": "Q1032",
    "Mali": "Q912",
    "Algérie ⵍⵣⵣⴰⵢⴻⵔ الجزائر": "Q262",
    "République démocratique du Congo": "Q974",
    "Congo": "Q971",
    "Cameroun": "Q1009",
    "السودان":"Q1049" #Sudan
}

df = pd.read_csv('Wikidata_information.csv', sep=";")

qs_lines = []

for _, row in df.iterrows():
    label = str(row["Label"]).strip()
    desc_de = str(row["Description"]).strip()
    desc_en = "archaeological site"
    type_qid = row["Type"].split()[0].strip()
    lat = row["Latitude"]
    lon = row["Longitude"]
    country = country_mapping.get(str(row["Country"]).strip(), None)
    #country = row["Country"]
    gazetteer_url = row["GazetteerID"]

    qs_lines.append("CREATE")
    qs_lines.append(f'LAST|Lde|"{label}"')
    qs_lines.append(f'LAST|Den|"{desc_en}"')
    qs_lines.append(f'LAST|Dde|"{desc_de}"')
    qs_lines.append(f'LAST|P31|{type_qid}')
    qs_lines.append(f'LAST|P625|@{lon}/{lat}|S854|"{gazetteer_url}"')
    if country:
        qs_lines.append(f'LAST|P17|{country}')

    qs_lines.append("")

# Save file
with open('quickstatements_ortsimport.txt', "w", encoding="utf-8") as f:
    f.write("\n".join(qs_lines))

Go to https://quickstatements.toolforge.org/

Log in with your Wikidata account.

Click on “New batch.”

If a message appears stating that the current wiki account does not have “autoconfirmed” status, 50 changes must be made manually in Wikidata. The account will then be automatically activated.

Paste the contents of the file or upload it.

Click on “Run” or “Only add” (for testing).

---

Gehe zu https://quickstatements.toolforge.org/

Melde dich mit deinem Wikidata-Account an.

Klicke auf „New batch“.

Wenn hier eine Meldung angezeigt wird, dass das aktuelle Wiki-Konto nicht den Status "autoconfirmed" hat, müssen noch manuell 50 Änderungen in Wikidata durchgeführt werden. Dann wird das Konto automatisch freigeschaltet.

Füge den Inhalt der Datei ein oder lade sie hoch.

Klicke auf „Run“ oder „Only add“ (zum Testen).

Developed by Lukas Lammers in the FAIR.rdm project, part of the DFG-funded SPP 2143 “Entangled Africa.”

With support from GitHub Copilot (AI assistant) for code development and optimization.

Version 1.0, July 15, 2025

---

Entwickelt von Lukas Lammers im Projekt FAIR.rdm, teil des DFG-geförderten SPP 2143 "Entangled Africa".

Mit Unterstützung von GitHub Copilot (KI-Assistent) bei der Code-Entwicklung und -Optimierung.

Version 1.0, 15.07.2025